In [33]:
import cv2

In [34]:
fileName = "test2"

In [35]:
mg_cv = cv2.imread(fileName + ".JPG")

In [36]:
gray=cv2.cvtColor(mg_cv,cv2.COLOR_BGR2GRAY)

In [37]:
import matplotlib.pyplot as plt

In [38]:
plt.imshow(gray, cmap='gray')

In [39]:
gray.shape

(2262, 4032)

In [40]:
from ctypes import cdll
test = cdll.LoadLibrary("./lib/siftdevice.so")

In [41]:
type(gray)

numpy.ndarray

In [42]:
import ctypes
import numpy as np

In [43]:
n = gray.shape[0]
m = gray.shape[1]

In [44]:
n,m

(2262, 4032)

In [45]:
res = np.empty((gray.shape[0], gray.shape[1]), dtype='float64')

In [46]:
gray = gray.astype('float64') # double

In [47]:
sigma = 10.0

In [48]:
kr = 16
ks = 4
ko = 8
S = 3
sigma_init = 0.5
contrast_threshold = 0.03
edge_response_threshold = 10.0
max_iterpolation = 10
time_arr4 = np.empty(4, dtype='float64')

In [49]:
res_num = 20
res_img = np.empty(gray.shape[0] * gray.shape[1] * 4 * res_num, dtype='float64')
res_n = np.empty(res_num, dtype='int32')
res_m = np.empty(res_num, dtype='int32')

# gray_t* gray_img, int n, int m, gray_t* res_img, int* res_n, int* res_m, int res_num, int S, double sigma_init
test.test.argtypes = [np.ctypeslib.ndpointer(dtype=gray.dtype, ndim=2, shape=gray.shape, flags='C_CONTIGUOUS'), 
                      ctypes.c_int,  # n
                      ctypes.c_int,  # m
                      np.ctypeslib.ndpointer(dtype=res_img.dtype, ndim=1, shape=res_img.shape, flags='C_CONTIGUOUS'), 
                      np.ctypeslib.ndpointer(dtype=res_n.dtype, ndim=1, shape=res_n.shape, flags='C_CONTIGUOUS'), 
                      np.ctypeslib.ndpointer(dtype=res_m.dtype, ndim=1, shape=res_m.shape, flags='C_CONTIGUOUS'), 
                      ctypes.c_int,  # res_num
                      ctypes.c_int,  # S
                      ctypes.c_double  # sigma_init
                     ]
test.test(gray, n, m, res_img, res_n, res_m, res_num, S, sigma_init)

cur = 0
show_all = True
for i in range(res_num):
    img = res_img[cur:cur+res_n[i] * res_m[i]]
    if show_all or ((i % (S + 3)) > 0 and (i % (S + 3)) < S + 1):
        plt.xlim([0, 2 * gray.shape[1]])
        plt.ylim([2 * gray.shape[0], 0])
        plt.imshow(img.reshape(res_n[i], res_m[i]), cmap='gray')
        #plt.savefig("dog{}.png".format(i))
        #plt.show()
    cur += res_n[i] * res_m[i]

print(res_img[res_n[0] * res_m[0]:res_n[0] * res_m[0]+res_n[1] * res_m[1]])

[1.73355016e+02 1.73359484e+02 1.73390354e+02 ... 1.37566795e+14
 1.37542892e+14 1.22945207e+14]


In [50]:
test.sift.argtypes = [np.ctypeslib.ndpointer(dtype=gray.dtype, ndim=2, shape=gray.shape, flags='C_CONTIGUOUS'), 
                          np.ctypeslib.ndpointer(dtype=res.dtype, ndim=2, shape=res.shape, flags='C_CONTIGUOUS'),
                          ctypes.c_int,  # n
                          ctypes.c_int,  # m
                          ctypes.c_int,  # kr
                          ctypes.c_int,  # ks
                          ctypes.c_int,  # ko
                          ctypes.c_int,  # S
                          ctypes.c_double,  # sigma_init
                          ctypes.c_double,  # contrast_threshold
                          ctypes.c_double,  # edge_response_threshold
                          ctypes.c_int,  # max_iterpolation
                          np.ctypeslib.ndpointer(dtype=time_arr4.dtype, ndim=1, shape=time_arr4.shape, flags='C_CONTIGUOUS')
                         ]
test.sift(gray, res, n, m, kr, ks, ko, S, 
          sigma_init, contrast_threshold, 
          edge_response_threshold, max_iterpolation, time_arr4)

0

In [51]:
time_arr4

array([ 2.70029497, 10.87547994,  1.13936114, 12.16437602])

In [52]:
n,m

(2262, 4032)

In [53]:
plt.imshow(res, cmap="gray")
plt.savefig(fileName+"_res.png")

In [54]:
res.shape

(2262, 4032)

In [55]:
import numpy as np

In [56]:
he = np.array(range(9), dtype='float64')
he[1] = he[3]
he[2] = he[6]
he[5] = he[7]
he_inv = np.array(range(9), dtype='float64')

In [57]:
# 0a1 1b1 2c1
# 3a2 4b2 5c2
# 6a3 7b3 8c3

In [58]:
det = he[0] * (he[4] * he[8] - he[5] * he[7]) \
        - he[3] * (he[1] * he[8] - he[2] * he[7]) \
        + he[6] * (he[1] * he[5] - he[2] * he[4])
he_inv[0] = (he[4] * he[8] - he[5] * he[7]) / det
he_inv[1] = (he[2] * he[7] - he[1] * he[8]) / det
he_inv[2] = (he[1] * he[5] - he[2] * he[4]) / det
he_inv[3] = (he[5] * he[6] - he[3] * he[8]) / det
he_inv[4] = (he[0] * he[8] - he[2] * he[6]) / det
he_inv[5] = (he[3] * he[2] - he[0] * he[5]) / det
he_inv[6] = (he[3] * he[7] - he[4] * he[6]) / det
he_inv[7] = (he[1] * he[6] - he[0] * he[7]) / det
he_inv[8] = (he[0] * he[4] - he[3] * he[1]) / det

In [59]:
print(he.reshape(3,3))

[[0. 3. 6.]
 [3. 4. 7.]
 [6. 7. 8.]]


In [60]:
print(he_inv.reshape(3,3))

[[-0.47222222  0.5        -0.08333333]
 [ 0.5        -1.          0.5       ]
 [-0.08333333  0.5        -0.25      ]]


In [61]:
import time

In [62]:
# https://docs.opencv.org/4.5.2/d7/d60/classcv_1_1SIFT.html
sift = cv2.SIFT_create(nOctaveLayers=S, contrastThreshold=contrast_threshold, 
                       edgeThreshold=edge_response_threshold, sigma=sigma_init)
t = time.time()
kp1, des1 = sift.detectAndCompute(mg_cv, None)   #des是描述子
t = time.time() - t 
img3 = cv2.drawKeypoints(mg_cv,kp1,mg_cv,color=(255,0,255))
plt.imshow(img3)

In [63]:
t

10.766781091690063

In [64]:
np.array([1]).dtype

dtype('int64')